In [2]:
import numpy as np
import pandas as pd

In [5]:
class AHP:
    def __init__(self, A, b):
        self.RI = (0, 0, 0.58, 0.9, 1.12, 1.24, 1.32, 1.41, 1.45, 1.49, 1.51)
        self.A = A
        self.b = b
        self.len_A = len(A)
        self.len_b = len(b[0])

    def cal_weights(self, input_matrix):
        input_matrix = np.array(input_matrix)
        n, n1 = input_matrix.shape
        assert n == n1, '不是一个方阵'
        
        eigenvalues, eigenvectors = np.linalg.eig(input_matrix)

        max_idx = np.argmax(eigenvalues)
        max_eigen = eigenvalues[max_idx].real
        eigen = eigenvectors[:, max_idx].real
        eigen = eigen / eigen.sum()

        CI = (max_eigen - n) / (n - 1)
        CR = CI / self.RI[n]
        return max_eigen, CR, eigen

    def run(self):
        k, CR, w0 = self.cal_weights(self.A)
        print('准则层：最大特征值{:<5f},CR={:<5f},'.format(k, CR),end='')
        if CR < 0.1:
            print('检验通过')
        else:
            print('检验不通过')
        print('准则层权重={}\n'.format(w0))

        klist, CR_list, wlist = [], [], []
        for i in self.b:
            k, CR, w = self.cal_weights(i)
            klist.append(k)
            CR_list.append(CR)
            wlist.append(w)

        pd_print = pd.DataFrame(wlist,
                                index=['准则' + str(i) for i in range(self.len_A)],
                                columns=['方案' + str(i) for i in range(self.len_b)],
                                )
        pd_print.loc[:, '最大特征值'] = klist
        pd_print.loc[:, 'CR'] = CR_list
        pd_print.loc[:, '一致性检验'] = pd_print.loc[:, 'CR'] < 0.1
        print('方案层')
        print(pd_print)

        # 目标层
        obj = np.dot(w0.reshape(1, -1), np.array(wlist))
        print('\n目标层', obj)
        print('最优选择是方案{}'.format(np.argmax(obj)))
        return obj


if __name__ == '__main__':

# 输入数据A与b

#     A = np.array([[1, 2, 7, 5, 5],
#                          [1 / 2, 1, 4, 3, 3],
#                          [1 / 7, 1 / 4, 1, 1 / 2, 1 / 3],
#                          [1 / 5, 1 / 3, 2, 1, 1],
#                          [1 / 5, 1 / 3, 3, 1, 1]])

#     b1 = np.array([[1, 1 / 3, 1 / 8], [3, 1, 1 / 3], [8, 3, 1]])
#     b2 = np.array([[1, 2, 5], [1 / 2, 1, 2], [1 / 5, 1 / 2, 1]])
#     b3 = np.array([[1, 1, 3], [1, 1, 3], [1 / 3, 1 / 3, 1]])
#     b4 = np.array([[1, 3, 4], [1 / 3, 1, 1], [1 / 4, 1, 1]])
#     b5 = np.array([[1, 4, 1 / 2], [1 / 4, 1, 1 / 4], [2, 4, 1]])

    b = [b1, b2, b3, b4, b5]
    a = AHP(A, b)
    a.run()

准则层：最大特征值5.072084,CR=0.014533,检验通过
准则层权重=[0.47583538 0.26360349 0.0538146  0.09806829 0.10867824]

方案层
          方案0       方案1       方案2     最大特征值            CR  一致性检验
准则0  0.081935  0.236341  0.681725  3.001542  8.564584e-04   True
准则1  0.595379  0.276350  0.128271  3.005535  3.075062e-03   True
准则2  0.428571  0.428571  0.142857  3.000000 -1.480297e-15   True
准则3  0.633708  0.191921  0.174371  3.009203  5.112618e-03   True
准则4  0.344545  0.108525  0.546931  3.053622  2.978976e-02   True

目标层 [[0.318586   0.23898522 0.44242878]]
最优选择是方案2
